## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%pip install tabulate pandas

In [ ]:
import pandas as pd

assert pd.__version__ >= "2.0.0", "Pandas version must be 2.0 or higher"
from utils import BankofCanadaRates, Transaction, Asset, Holdings
from IPython.display import display, HTML
from decimal import Decimal


def isclose(a, b, rel_tol=Decimal("1e-9"), abs_tol=Decimal("0")):
    a, b = Decimal(a), Decimal(b)
    if a == b:
        return True
    diff = abs(a - b)
    return diff <= abs_tol or diff <= rel_tol * max(abs(a), abs(b))


def displayPandas(df, precision=10, text=False):
    # styled_df = df.style.format(lambda x: f'{x:.{precision}g}' if isinstance(x, Decimal) else x)
    # display(HTML(styled_df.to_html()))

    df = df.copy()
    for col in df.columns:
        if df[col].apply(lambda x: isinstance(x, Decimal)).any():
            df[col] = df[col].apply(
                lambda x: f"{x:.{precision}g}" if isinstance(x, Decimal) else x
            )

    if text:
        return df.to_markdown()
    else:
        display(HTML(df.to_html()))

In [ ]:
transactions_filepath = "transactions.csv"

decimal_columns = ["quantity", "price", "fees"]

trxs_log = pd.read_csv(transactions_filepath, dtype=str)
trxs_log.columns = map(lambda x: x.lower().replace(" ", "_"), trxs_log.columns)
trxs_log.index.name = "id"
# Fill in missing values
trxs_log = trxs_log.sort_values(["date", "id"]).fillna({"description": "", "fees": 0, "note": ""})
# Convert numeric columns to Decimal
for col in decimal_columns:
    trxs_log[col] = trxs_log[col].apply(lambda x: Decimal(x.replace(",", "")) if x else Decimal(0))

displayPandas(trxs_log.tail(5))

assert trxs_log.isnull().sum().sum() == 0
trxs = trxs_log.apply(lambda r: Transaction(**r), axis=1).tolist()

## Process

In [ ]:
import json
from dataclasses import asdict

holdings = Holdings()
capgains = []
boc = BankofCanadaRates(start_date="2018-01-01")
reporting_currency = "CAD"

# artificial initial cash balance
holdings.add(
    Asset(trxs_log["date"].min(), reporting_currency, quantity=50000, acb=1)
)


def process_transaction(trx: Transaction):
    # reflect fees in the price
    trx = trx.with_effective_price()

    # Determine the quote_to_reporting_rate
    if trx.quote_to_reporting_rate is None:
        try:
            if trx.quote_currency == reporting_currency:
                trx.quote_to_reporting_rate = 1
            else:
                # retrieve the exchange rate from Bank of Canada
                trx.quote_to_reporting_rate = boc.get_rate(
                    trx.quote_currency, reporting_currency, trx.date
                )
        except:
            raise Exception(
                f"Error getting rate for {trx.date} {trx.quote_currency} to {reporting_currency}"
            )

    # Vesting transactions are funded by the company, so we need to add
    # a preceding funding transaction
    if "Vest" in trx.description:
        process_transaction(
            Transaction(
                date=trx.date,
                description=trx.description.replace("Vest", "Funding"),
                base_currency=trx.quote_currency,
                quote_currency=reporting_currency,
                quantity=trx.cost,
                price=trx.quote_to_reporting_rate,
                fees=0,
                quote_to_reporting_rate=1,
            )
        )

    # Flip the transaction if it is a sell order
    if trx.quantity < 0:
        trx = trx.flip()

    # Get the current holdings
    base_holding = holdings.get(trx.base_currency, trx.date)
    quote_holding = holdings.get(trx.quote_currency, trx.date)

    # Update the ACB and quantity for the base currency
    if trx.base_currency != reporting_currency:
        base_holding.acb = (
            base_holding.quantity * base_holding.acb
            + trx.cost * quote_holding.acb
        ) / (base_holding.quantity + trx.quantity)
    base_holding.quantity += trx.quantity
    base_holding.date = trx.date

    # Calculate the capital gain for liquidating transactions
    if trx.base_currency == reporting_currency:
        cost_base = quote_holding.acb * trx.cost
        gross_proceeds = trx.quantity
        capital_gain = gross_proceeds - cost_base
        capgains.append(
            {
                "Date": trx.date,
                "Cost Base": cost_base,
                "Gross Proceeds": gross_proceeds,
                "Capital Gain": capital_gain,
            }
        )

    # Update the quantity for the quote currency
    if (quote_holding.quantity < trx.cost) and (
        not isclose(quote_holding.quantity, trx.cost)
    ):
        raise Exception(
            f"Insufficient funds to complete transaction on {trx.date}.\n"
            f" - Cost: {trx.cost} {trx.quote_currency}\n"
            f" - Current holdings: {quote_holding.quantity} {trx.quote_currency}\n"
            "Details:\n"
            f"Transaction: {json.dumps(asdict(trx), indent=4)}\n"
            f"Current holdings:\n"
            f"{displayPandas(holdings.current, text=True)}\n"
        )
    quote_holding.quantity -= trx.cost
    quote_holding.date = trx.date

    holdings.add(base_holding, overwrite=True)
    holdings.add(quote_holding, overwrite=True)


for trx in trxs:
    process_transaction(trx)

## Status

In [ ]:
df_capgains = pd.DataFrame(capgains)
df_capgains["Date"] = pd.to_datetime(df_capgains["Date"])
print("Capital Gains")
displayPandas(df_capgains)

for year in range(df_capgains["Date"].dt.year.min(), df_capgains["Date"].dt.year.max() + 1):
    print(f"Capital Gains: {year}")
    displayPandas(
        df_capgains.query(f"'{year}-01-01' <= Date <= '{year}-12-31'")
        .drop(columns="Date")
        .sum()
        .to_frame()
        .T
    )

print("Holdings")
displayPandas(holdings.current)

## Holdings Time Series

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots

fig = px.line(holdings.df, x="date", y="quantity", color="asset", markers=True)
fig2 = px.line(holdings.df, x="date", y="acb", color="asset")
fig2.update_traces(yaxis="y2", line=dict(dash="dashdot"))

subfig = make_subplots(specs=[[{"secondary_y": True}]])
subfig.add_traces(fig.data + fig2.data)
subfig.layout.xaxis.title="Date"
subfig.layout.yaxis2.title="ACB (CAD)" + " --"
subfig.layout.yaxis.title="Quantity"
subfig.show()